# Test snippets

In [1]:
# import the libraries
import telnetlib, datetime, time, csv
import numpy as np
import pyrpl
from pathlib import Path

timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
path = Path.home().as_posix()
full_path = Path(path+'/Dropbox (CoQuMa)/LabNotes/NaKa/'+timestamp[:7]+'/'+timestamp[:10]+'/')

## Ramp the laser power according to an array of voltages 

In [ ]:
# set up the ramping details and run the cell below
start = 1 # in V
stop = 3
step = 0.1
ramp = np.arange(start, stop+step, step)

# set the duration of the ramping steps or...
delay = 10 # in seconds
total_time = len(ramp) * delay

# ...the total ramping time
# total_time = 600 # in seconds
# delay = total_time / len(ramp)

In [ ]:
session = telnetlib.Telnet('10.0.2.214')        # start telnet session

for voltage in ramp:
    session.write(bytes('ls_tool edfa_set_phdout edfa1 ' + str(voltage) + '\n', 'utf-8'))
    session.read_until(b'pid_setpoint = ')
    setpoint = session.read_until(b'\n').decode('utf-8')
    
    EDFA = float(setpoint) * 2.999725 / 39321.6
                
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    data = {'updated' : timestamp }
    data['EDFA'] = np.round(EDFA,6)
    
    try :
        with open(str(full_path)+'\\klaser'+'_'+timestamp[:10]+'.csv', 'a',newline='', encoding='UTF8') as f:
            writer = csv.writer(f)
            writer.writerow([value for key, value in data.items()])
            f.close()
    except FileNotFoundError : 
        with open(Path.cwd().as_posix()+'\\data\\klaser'+'_'+timestamp[:10]+'.csv','a',newline='', encoding='UTF8') as f:
            writer = csv.writer(f)
            writer.writerow([value for key, value in data.items()])
            f.close()
            
    step_time = 0
    while step_time < delay:
        print('Current setpoint : ' + str(np.round(voltage,3)) + ' V -- Time remaining : ' + str(total_time).rjust(3) + ' s', end='\r')
        total_time -= 1
        step_time += 1
        time.sleep(1)

session.write(b'exit\n')
session.close()

## Detune the laser light with setting the PID at the Redpitaya

In [2]:
# connect to the redpitaya and create Redpitaya instance
p = pyrpl.Pyrpl(hostname='10.0.2.111', config='111.yml'); r = p.rp

INFO:pyrpl:All your PyRPL settings will be saved to the config file
    C:\Users\NaKaControl\pyrpl_user_dir\config\111.yml
If you would like to restart PyRPL with these settings, type "pyrpl.exe 111" in a windows terminal or 
    from pyrpl import Pyrpl
    p = Pyrpl('111')
in a python terminal.
INFO:pyrpl.redpitaya:Successfully connected to Redpitaya with hostname 10.0.2.111.


In [ ]:
print(r.pid0.setup())

In [3]:
# shortcut to save data
def save_data(data):
    path = Path.home().as_posix()
    full_path = Path(path+'/Dropbox (CoQuMa)/LabNotes/NaKa/'+timestamp[:7]+'/'+timestamp[:10]+'/')
    
    try:
        with open(str(full_path)+'\\redpitaya_'+timestamp[:10]+'.csv', 'a',newline='', encoding='UTF8') as f:
            writer = csv.writer(f)
            writer.writerow(data)
    except FileNotFoundError :
        with open(Path.cwd().as_posix()+'\\data\\redpitaya_'+timestamp[:10]+'.csv', 'a',newline='', encoding='UTF8') as f:
            writer = csv.writer(f)
            writer.writerow(data)

### Saving data only

In [ ]:
# just keep track of the lock...
while True:
    now = datetime.datetime.now()
    timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
    
    data = [ timestamp, r.scope.voltage_in1, r.pid0.setpoint ]
    save_data(data)
    
    time.sleep(5)

### Doing the actual ramping/measurement

In [ ]:
# create an array of detuning and run the cell below
start = -0.15 # in V
stop = -0.05
step = 0.01
ramp = np.arange(start, stop+step, step)

# set the duration of the ramping steps or...
delay = 30 # in seconds
total_time = len(ramp) * delay

# ...the total ramping time
# total_time = 600 # in seconds
# delay = total_time / len(ramp)

In [ ]:
restore = r.pid0.setpoint

# go through the ramp
for setpoint in ramp:
    r.pid0.setpoint = setpoint

    step_time = 0
    while step_time < delay:
        now = datetime.datetime.now()
        timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
        
        data = [ timestamp, r.scope.voltage_in1 ]
        save_data(data)
        
        print('Current setpoint : ' + str(setpoint) + ' V -- Time remaining : ' + str(total_time).rjust(3) + ' s', end='\r')
        total_time -= 1
        step_time += 1
        time.sleep(1)
        
# restore previous settings
r.pid0.setpoint = restore

### Include the SlackBot here...?

In [ ]:
if -1 > r.scope.input1 || r.scope.voltage_in1 > 1.0:
    redirect_url = "https://hydra.kip.uni-heidelberg.de:8443/f21server/#/"
    text = ":fire: ALERT!!! \n The laser is out of lock! Check the NaKaControl"
    channel_id = "C02F1MWPLM7" # naka_laserlock channel
    url = "https://slack.com/api/chat.postMessage"
    payload = {
        "channel": "C02F1MWPLM7",
        #"text": "",
        "attachments": [{
            "text": text,
            "color": "#ad1721",
            "attachment_type": "default",
            "actions": [
                {
                    "name": "Take me there!",
                    "text": "Take me there!",
                    "type": "button",
                    "style": "primary",
                    "url": redirect_url
                },
            ]
            }]
        }
    headers = { 
        "Content-type" : "application/json",
        "Authorization" : "Bearer " + Slackbot.bot_token,
    }
    
    try:
        r = requests.post(url, json=payload, headers=headers)
        r.raise_for_status()
        print(r.text)